In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes(balanced=True)
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.60)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 5.31 sec - Loss 0.063312 - ACC 57.05% - ACC Mean 57.05% - AUC 59.01% - AUC Mean 59.01% - Deter 000
Ite 00050 - 1.30 sec - Loss 0.011089 - ACC 56.75% - ACC Mean 56.74% - AUC 58.61% - AUC Mean 59.60% - Deter 024
Ite 00100 - 1.32 sec - Loss 0.005408 - ACC 55.95% - ACC Mean 57.47% - AUC 59.01% - AUC Mean 60.29% - Deter 019
Ite 00131 - 1.11 sec - Loss 0.004104 - ACC 57.25% - ACC Mean 57.53% - AUC 60.42% - AUC Mean 60.17% - Deter 050
Early stop ite 131, rollback to correction of ite 81, whith acc of 59.5% and auc of 62.55%
OUR METHOD RUN 1/10 - acc: 59.5% +- 0.0% - auc: 62.55% +- 0.0%
Ite 00000 - 1.06 sec - Loss 0.113753 - ACC 57.70% - ACC Mean 57.70% - AUC 61.20% - AUC Mean 61.20% - Deter 000
Ite 00050 - 1.17 sec - Loss 0.038747 - ACC 55.30% - ACC Mean 58.01% - AUC 59.01% - AUC Mean 61.06% - Deter 044
Ite 00056 - 1.13 sec - Loss 0.035359 - ACC 58.60% - ACC Mean 57.97% - AUC 61.27% - AUC Mean 60.95% - Deter 050
Early stop ite 56, rollback to correction of ite 6, whith acc of 59.9

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 60.45% +- 0.0% - auc: 63.55% +- 0.0%
MEAN - acc: 60.1% +- 0.0% - auc: 63.08% +- 0.0%
KNN - acc: 57.05% +- 0.0% - auc: 58.81% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [01:04<00:00, 77.72it/s]



GAIN RUN 1/10 - acc: 56.75% +- 2.2% - auc: 59.53% +- 2.3%



100%|██████████| 5000/5000 [01:12<00:00, 69.40it/s]



GAIN RUN 2/10 - acc: 57.95% +- 0.91% - auc: 60.6% +- 1.05%



100%|██████████| 5000/5000 [01:10<00:00, 71.22it/s]


...


GAIN RUN 8/10 - acc: 57.2% +- 4.16% - auc: 60.29% +- 4.14%



100%|██████████| 5000/5000 [01:15<00:00, 65.93it/s]



GAIN RUN 9/10 - acc: 56.55% +- 2.2% - auc: 60.21% +- 3.48%



100%|██████████| 5000/5000 [01:07<00:00, 74.17it/s]



GAIN RUN 10/10 - acc: 56.5% +- 1.72% - auc: 58.34% +- 2.15%

GAIN GLOBAL - acc: 57.32% +- 0.7% - auc: 60.19% +- 0.88%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 57.65% +- 0.0% - auc: 60.8% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1

MISSFOREST RUN 1/10 - acc: 57.1% +- 2.07% - auc: 61.38% +- 2.89%

Iteration: 0
Iteration: 1

MISSFOREST RUN 2/10 - acc: 58.0% +- 1.72% - auc: 59.81% +- 1.89%

Iteration: 0
Iteration: 1

MISSFOREST RUN 3/10 - acc: 59.25% +- 2.73% - auc: 62.17% +- 2.64%

Iteration: 0
Iteration: 1

MISSFOREST RUN 4/10 - acc: 57.2% +- 0.84% - auc: 59.29% +- 2.35%

Iteration: 0
Iteration: 1

MISSFOREST RUN 5/10 - acc: 58.1% +- 1.71% - auc: 61.14% +- 2.1%

Iteration: 0
Iteration: 1

MISSFOREST RUN 6/10 - acc: 57.5% +- 2.4% - auc: 60.39% +- 3.48%

Iteration: 0
Iteration: 1

MISSFOREST RUN 7/10 - acc: 56.8% +- 1.93% - auc: 58.31% +- 1.89%

Iteration: 0
Iteration: 1

MISSFOREST RUN 8/10 - acc: 56.75% +- 1.13% - auc: 59.58% +- 2.3%

Iteration: 0
Iteration: 1

MISSFOREST RUN 9/10 - acc: 56.4% +- 2.6% - auc: 59.4% +- 1.97%

Iteration: 0
Iteration: 1

MISSFOREST RUN 10/10 - acc: 58.45% +- 1.91% - auc: 61.29% +- 3.14%


MISSFOREST GLOBAL - acc: 57.56% +- 0.84% - auc: 60.28% +- 1.14%


## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 58.15% +- 1.99% - auc: 61.45% +- 2.12%
SOFTIMPUTE RUN 2/10 - acc: 59.05% +- 1.81% - auc: 61.64% +- 2.48%
SOFTIMPUTE RUN 3/10 - acc: 57.1% +- 1.85% - auc: 60.18% +- 1.69%
SOFTIMPUTE RUN 4/10 - acc: 57.2% +- 3.05% - auc: 61.17% +- 3.31%
SOFTIMPUTE RUN 5/10 - acc: 57.3% +- 3.1% - auc: 61.08% +- 3.28%
SOFTIMPUTE RUN 6/10 - acc: 58.05% +- 1.55% - auc: 59.44% +- 2.1%
SOFTIMPUTE RUN 7/10 - acc: 55.55% +- 3.17% - auc: 57.45% +- 3.05%
SOFTIMPUTE RUN 8/10 - acc: 57.8% +- 3.06% - auc: 61.77% +- 3.3%
SOFTIMPUTE RUN 9/10 - acc: 58.85% +- 3.04% - auc: 61.23% +- 3.19%
SOFTIMPUTE RUN 10/10 - acc: 56.5% +- 1.66% - auc: 59.04% +- 2.44%

SOFTIMPUTE GLOBAL - acc: 57.56% +- 1.01% - auc: 60.44% +- 1.33%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.9455

SINKHORN RUN 1/10 - acc: 57.75% +- 3.12% - auc: 60.99% +- 3.58%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.9396

SINKHORN RUN 2/10 - acc: 55.85% +- 2.94% - auc: 58.35% +- 3.46%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.8754

SINKHORN RUN 3/10 - acc: 57.35% +- 4.05% - auc: 60.32% +- 4.33%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 4.2301

SINKHORN RUN 4/10 - acc: 58.65% +- 2.0% - auc: 61.51% +- 0.69%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 4.0121

SINKHORN RUN 5/10 - acc: 57.3% +- 2.69% - auc: 59.95% +- 3.74%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 4.0531

SINKHORN RUN 6/10 - acc: 57.75% +- 1.93% - auc: 59.9% +- 2.1%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 3.9835

SINKHORN RUN 7/10 - acc: 57.7% +- 2.91% - auc: 60.3% +- 2.86%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 4.0190

SINKHORN RUN 8/10 - acc: 56.85% +- 1.33% - auc: 59.99% 

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 58.1% +- 3.27% - auc: 60.49% +- 3.55%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 57.1% +- 3.32% - auc: 59.22% +- 3.2%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 58.9% +- 2.56% - auc: 62.07% +- 1.67%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 56.45% +- 2.63% - auc: 59.63% +- 4.35%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 57.55% +- 2.57% - auc: 60.37% +- 3.82%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 59.2% +- 1.98% - auc: 61.23% +- 2.69%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 59.85% +- 2.07% - auc: 62.48% +- 1.32%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 59.05% +- 3.17% - auc: 62.35% +- 3.85%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 5

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is even with MEDIAN
Metric AUC - OUR METHOD is even with MEDIAN
Metric ACC - OUR METHOD is even with MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is significantly better than SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly better than MIDA
